<a href="https://colab.research.google.com/github/LauraGuio/MachineLearning/blob/master/Clasificador_ukelele_trompeta_piano_JLN_LSGC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Clasificador de instrumentos de cuerda (Trompeta, Piano, Ukelele)

##1.Importar librerías

In [11]:
!pip install pyprind

In [12]:
import requests
import os
import sys
import tarfile
import time
import pyprind

from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K

##2.Obtener DataSet

###2.1 Avance de proceso en background

In [13]:
def reporthook(count, block_size, total_size):
  global start_time
  if count == 0:
    start_time = time.time()
    return 
  duration = time.time() - start_time
  progress_size = int(count * block_size)
  speed = progress_size / (1024.**2 * duration)
  percent = count * block_size * 100. / total_size
  sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos trancurrido" %
                   (percent, progress_size / (1024.**2), speed, duration))
  sys.stdout.flush()

###2.2 Descargando .zip de GitHub

In [ ]:
source = 'https://github.com/LauraGuio/MachineLearning/blob/master/dataset.zip'
target = 'dataset.zip'

if not os.path.isfile('dataset.zip'):

  if (sys.version_info < (3,0)):
    import urllib
    urllib.urlretrieve(source, target, reporthook)

  else:
    import urllib.request
    urllib.request.urlretrieve(source, target, reporthook)

###2.3 Extraemos los archivos

In [14]:
from zipfile import ZipFile
with ZipFile ('/content/drive/MyDrive/DeepLearning/dataset.zip', 'r') as zip:
  zip.extractall()
  print('Fin')

Fin


In [ ]:
import zipfile
zipfilename = ["datos.zip"]
for filename in zipfilename:
    print(str(filename) + " " + str(zipfile.is_zipfile(filename)))

datos.zip False


##3. Limpiamos sesión de Keras

In [15]:
K.clear_session()
data_entrenamiento = '/content/dataset/entrenamiento'
data_validacion = '/content/dataset/validacion'

###3.1 Definimos características de red neuronal

In [16]:
epocas=1200
longitud, altura = 150, 150 #redimensionar imágenes
batch_size = 32  #Imágenes por iteración
pasos = 30  #1000 pasos por iteración
validation_steps = 1000
filtrosConv1 = 32
filtrosConv2 = 64 
filtrosConv3=128
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_filtro3 = (1,1)
tamano_pool = (2, 2)
clases = 3  #3 clases que son piano, trompeta y ukelele 
lr = 0.0004 

##4. Preparar imágenes de Train y Test

In [17]:
entrenamiento_datagen = ImageDataGenerator(
    rotation_range = 40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1. / 255)

###4.1 Preparar imágenes

In [18]:
entrenamiento_gen = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 1346 images belonging to 3 classes.
Found 269 images belonging to 3 classes.


##5. Crear red convulocional

In [19]:
cnn = Sequential()

cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv3, tamano_filtro3, padding="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

cnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

##6. Entrenamos

In [20]:
cnn.fit_generator(
    entrenamiento_gen,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

Epoch 1/1200
30/30 [==============================] - 36s 1s/step - loss: 1.2920 - accuracy: 0.4237 - val_loss: 0.5233 - val_accuracy: 0.6320
Epoch 2/1200
30/30 [==============================] - 31s 1s/step - loss: 0.7154 - accuracy: 0.5280
Epoch 3/1200
30/30 [==============================] - 30s 1s/step - loss: 0.5274 - accuracy: 0.5989
Epoch 4/1200
30/30 [==============================] - 31s 1s/step - loss: 0.4965 - accuracy: 0.6473
Epoch 5/1200
30/30 [==============================] - 31s 1s/step - loss: 0.4665 - accuracy: 0.6624
Epoch 6/1200
30/30 [==============================] - 31s 1s/step - loss: 0.4734 - accuracy: 0.6763
Epoch 7/1200
30/30 [==============================] - 30s 1s/step - loss: 0.4138 - accuracy: 0.7323
Epoch 8/1200
30/30 [==============================] - 30s 1s/step - loss: 0.4160 - accuracy: 0.7312
Epoch 9/1200
30/30 [==============================] - 31s 1s/step - loss: 0.4123 - accuracy: 0.7247
Epoch 10/1200
30/30 [==============================] - 32s

##7. Guardar el modelo

In [22]:
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo_LSGC_JLN.h5')
cnn.save_weights('./modelo/peso_LSGC_JLN.h5')

###7.1 Descargamos modelo

In [ ]:
from google.colab import files 
files.download('./modelo/modelo_LSGC_JLN.h5')
files.download('./modelo/peso_LSGC_JLN.h5')

##8. Prueba del modelo

In [23]:
import numpy as np
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.models import load_model
import h5py

In [27]:
lonitud, altura = 150, 150
modelo='./modelo/modelo_LSGC_JLN.h5'
pesos_modelo = './modelo/peso_LSGC_JLN.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos_modelo)

In [28]:
def predict(file):
  x = load_img(file, target_size=(longitud, altura))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = cnn.predict(x)
  result=array[0]
  answer = np.argmax(result)
  class_names=['piano','trompeta','ukelele']
  print(class_names[answer])
  return answer

**Predicciones**

In [29]:
predict('/content/[www.google.com][31]210872b.jpg')

piano


0

In [30]:
predict('/content/2.jpg')

trompeta


1

In [31]:
predict('/content/3.jpg')

trompeta


1

In [33]:
predict('/content/5.jpg')

ukelele


2